# Lesson I

## Real-world Examples

You've learned a lot about how decorators work. This lesson will walk you through some real-world decorators so that you can start to recognize common decorator patterns.

### Time a function

The ``timer()`` decorator runs the decorated function and then prints how long it took for the function to run. I usually wind up adding some version of this to all of my projects because it is a pretty easy way to figure out where your computational bottlenecks are. 

```python
    import time

    def timer(func):
        """A decorator that prints how long a function took to run.

        Args:
            func (callable): The function being decorated.

        Returns:
            callable: The decorated function.
        """    
```

All decorators have fairly similar-looking docstrings because they all take and return a single function. For brevity, I will only include the description of the function in the docstrings of the examples that follow.



In [6]:
import time

def timer(func):
    """A decorator that prints how long a function took to run."""
    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        # When wrapper() is called, get the current time
        t_start = time.time()
        # Call the decorated function and store the result.
        result = func(*args, **kwargs)
        # Get the total time it took to run, and print it.
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
        return result
    return wrapper
    

Like most decorators, we'll start off by defining a ``wrapper()`` function. This is the function that the decorator will return. ``wrapper()`` takes any number of positional and keyword arguments so that it can be used to decorate any function. 

The first thing the new function will do is record the time that it was called with the ``time()`` function. Then ``wrapper()`` gets the result of calling the decorated function. We don't return that value yet though. 

After calling the decorated function, ``wrapper()`` checks the time again, and prints a message about how long it took to run the decorated function. Once we've done that, we need to return the value that the decorated function calculated.

In [7]:
@timer
def sleep_n_seconds(n):
    time.sleep(n)
    
print(sleep_n_seconds(5))

print(sleep_n_seconds(10))    

sleep_n_seconds took 5.011141538619995s
None
sleep_n_seconds took 10.015560150146484s
None


So if we decorate this simple ``sleep_n_seconds()`` function, you can see that sleeping for 5 seconds takes about 5 seconds, and sleeping for 10 seconds takes about 10 seconds. This is a trivial use of the decorator to show it working, but it can be very useful for finding the slow parts of your code.

## Memoizing

Memoizing is the process of storing the results of a function so that the next time the function is called with the same arguments; you can just look up the answer.

In [8]:
def memoize(func):
    """Store the result of the decorated function for fast lookup
    """
    # Store results in a dict that maps arguments to results
    cache = {}
    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        # If these arguments haven't been seen before,
        if (args, kwargs) not in cache:
            # Call func() and store the result.
            cache [(args, kwargs)] = func(*args, **kwargs)
        return cache[(args, kwargs)]
    return wrapper    

We start by setting up a dictionary that will map arguments to results. Then, as usual, we create ``wrapper()`` to be the new decorated function that this decorator returns. 

When the new function gets called, we check to see whether we've ever seen these arguments before. If we haven't, we send them to the decorated function and store the result in the ``"cache"`` dictionary. 

Now we can look up the return value quickly in a dictionary of results. The next time we call this function with those same arguments, the return value will already be in the dictionary.

In [ ]:
@memoize
def slow_function(a, b):
    print('Sleeping...')
    time.sleep(5)
    return a + b

slow_function(3, 4)

Here we are memoizing ``slow_function()``. ``slow_function()`` simply returns the sum of its arguments. In order to simulate a slow function, we have it sleep for 5 seconds before returning. 

If we call ``slow_function()`` with the arguments *3 and 4, it will sleep for 5 seconds and then return 7*. But if we call ``slow_function()`` with the arguments 3 and 4 again, it will immediately return 7. Because we've stored the answer in the cache, the decorated function doesn't even have to call the original ``slow_function()`` function.

## When to use decorators

So when is it appropriate to use a decorator? You should consider using a decorator when you want to add some common bit of code to multiple functions. 

* Add common behavior to multiple functions

```python
    @timer
    def foo():
        # Do some computation

    @timer
    def bar():
        # Do some computation

    @timer
    def baz():
        # Do some computation        
```

We could have added timing code in the body of all three of these functions, but that would violate the principle of Don't Repeat Yourself. Adding a decorator is a better choice.

# Lesson II

## Decorators and Metadata

One of the problems with decorators is that they obscure the decorated function's metadata. In this lesson, I'll show you why it's a problem and how to fix it.

Here we have a nice function, ``sleep_n_seconds()``, with a *docstring* that explains exactly what it does. If we look at the ``docstring`` attribute, we can see the text of the *docstring*.

In [ ]:
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
    
    Args:
        n (int) : The number of seconds to pause for.
    """
    time.sleep(n)
    
print(sleep_n_seconds.__doc__)    

Pause processing for n seconds.
    
    Args:
        n (int) : The number of seconds to pause for.
    


We can also access other metadata for the function, like its name and default arguments.

In [ ]:
print(sleep_n_seconds.__name__)

print(sleep_n_seconds.__defaults__)

sleep_n_seconds
(10,)


But watch what happens when we decorate ``sleep_n_seconds()`` with the ``timer()`` decorator as we've done here. 

When we try to *print* the *docstring*, we get *nothing back*. Even stranger, when we try to look up the function's name, Python tells us that ``sleep_n_seconds()``'s name is ``"wrapper"``.

In [10]:
@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
    
    Args:
        n (int) : The number of seconds to pause for.
    """
    time.sleep(n)
    
print(sleep_n_seconds.__doc__)

print(sleep_n_seconds.__name__)    

None
wrapper


To understand why we have to examine the ``timer()`` decorator. 

```python
    def timer(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f"{func.__name__} took {end - start} seconds")
            return result
        return wrapper
```

Remember that when we write decorators, we almost always define a nested function to return. Because the decorator overwrites the ``sleep_n_seconds()`` function, when you ask for ``sleep_n_seconds()``'s docstring or name, you are actually referencing the nested function that was returned by the decorator. In this case, the nested function was called ``wrapper()`` and it didn't have a docstring.

## functools.wraps()

Fortunately, Python provides us with an easy way to fix this. The ``wraps()`` function from the ``functools`` module is a decorator that you use when defining a decorator. 

If you use it to decorate the wrapper function that your decorator returns, it will modify ``wrapper()``'s metadata to look like the function you are decorating.

In [11]:
from functools import wraps
def timer(func):
    """A decorator that prints how long a function took to run"""
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        t_start = time.time()
        
        result = func(*args, **kwargs)
        
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
        
        return result
    return wrapper



Notice that the ``wraps()`` decorator takes the function you are decorating as an argument.

If we use this updated version of the ``timer()`` decorator to decorate ``sleep_n_seconds()`` and then try to print ``sleep_n_seconds()``'s docstring, we get the result we expect.

In [14]:
@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
    
    
    Args:
        n (int) : The number of seconds to pause for.
    """
    time.sleep(n)
    
    
print(sleep_n_seconds.__doc__)   

Pause processing for n seconds.
    
    
    Args:
        n (int) : The number of seconds to pause for.
    


Likewise, printing the name or any other metadata now gives you the metadata from the function being decorated rather than the metadata of the ``wrapper()`` function.

In [15]:
print(sleep_n_seconds.__name__)

print(sleep_n_seconds.__defaults__)

sleep_n_seconds
None


### Access to original function

As an added bonus, using ``wraps()`` when creating your decorator also gives you easy access to the original undecorated function via the ``__wrapped__`` attribute. Of course, you always had access to this function via the closure, but this is an easy way to get to it if you need it.

In [16]:
print(sleep_n_seconds.__wrapped__)

<function sleep_n_seconds at 0x00000200221A6700>


# Lesson III

## Decorators that take arguments

Sometimes it would be nice to add arguments to our decorators. To do that, we need another level of function nesting.

In [18]:
def run_three_times(func):
    def wrapper(*args, **kwargs):
        for i in range(3):
            func(*args, **kwargs)
    return wrapper

@run_three_times
def print_sum(a, b):
    print(a + b)
    
print_sum(3, 5)            

8
8
8


Let's consider this silly ``run_three_times()`` decorator. If you use it to decorate a function, it will run that function three times. 
If we use it to decorate the ``print_sum()`` function and then run ``print_sum(3,5)``, it will print 8 three times.

Let's think about what we would need to change if we wanted to write a ``run_n_times()`` decorator. We want to pass ``"n"`` as an argument, instead of hard-coding it in the decorator.

```python
    def run_n_times(func):
        def wrapper(*args, **kwargs):
            # How do we pass 'n' into this function?
            for i in range(???):
                func(*args, **kwargs)
        return wrapper
```               

If we had some way to pass n into the decorator, we could decorate ``print_sum()`` so that it gets run three times and decorate ``print_hello()`` to run five times.

```python
    @run_n_times(3)
    def print_sum(a, b):
        print(a + b)

    @run_n_times(5)
    def print_hello():
        print("Hello")    
```

But a decorator is only supposed to take one argument - the function it is decorating. Also, when you use decorator syntax, you're not supposed to use the parentheses. So what gives?

### A decorator factory

To make ``run_n_times()`` work, we have to turn it into a function that returns a decorator, rather than a function **that is** a decorator. 

So let's start by redefining ``run_n_times()`` so that it takes ``n`` as an *argument*, instead of ``func``. Then, inside of ``run_n_times()``, we'll define a new *decorator* function... 

In [19]:
def run_n_times(n):
    """Define and return a decorator"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

        

This function takes ``"func"`` as an argument because it is the function that will be acting as our decorator. 

We start our new decorator with a nested ``wrapper()`` function, as usual. Now, since we are still inside the ``run_n_times()`` function, we have access to the ``n`` parameter that was passed to ``run_n_times()``. 

We can use that to control how many times we repeat the loop that calls our decorated function. As usual for any decorator, we return the new ``wrapper()`` function. And, if ``run_n_times()`` returns the ``decorator()`` function we just defined, then we can use that return value as a decorator. 


In [20]:
@run_n_times(3)
def print_sum(a, b):
    print(a + b)
    
print_sum(3, 5)    

8
8
8


To prove to you that it works the way we expect here is ``print_sum()`` decorated with ``run_n_times(3)``. When we call ``print_sum()`` with the arguments *3* and *5*, it prints *8* *three* times. 

In [21]:
@run_n_times(5)
def print_hello():
    print('Hello!')
    
print_hello()    

Hello!
Hello!
Hello!
Hello!
Hello!


And we can just as easily decorate ``print_hello()``, which prints a hello message, with ``run_n_times(5)``. When we call ``print_hello()``, we get *five* hello messages, as expected.

# Lesson IV

## Timeout(): a real world example

We're going to finish up by looking at an example of a real-world decorator that takes an argument to give you a better sense of how they work.

For our first example, let's imagine that we have some functions that occasionally either run for longer than we want them to or just hang and never return.

```python
    def function1():
        # This function sometimes
        # runs for a loooooong time


    def function2():
        # This function sometimes
        # hangs and never returns    
```

It would be nice if we could add some kind of ``timeout()`` decorator to those functions that will raise an error if the function runs for longer than expected.


### Timeout - Background info

To create the ``timeout()`` decorator, we are going to use some functions from Python's ``signal`` module. These functions have nothing to do with decorators, but understanding them will help you understand the ``timeout()`` decorator.

```python
    import signal
    def raise_timeout(*args, **kwargs):
        raise TimeoutError        
```

The ``raise_timeout()`` function simply raises a *TimeoutError* when it is called.

```python
    # When an 'alarm' signal goes off, call raise_timeout()
    signal.signal(signnalum=signal.SIGALRM, handler=raise_timeout)
```

The ``signal()`` function tells Python, *"When you see the signal whose number is signalnum, call the handler function."* In this case, we tell Python to call ``raise_timeout()`` whenever it sees the alarm signal. 

```python
    # Set off an alarm in 5 seconds
    signal.alarm(5)

    # Cancel the alarm
    signal.alarm(0)
```

The ``alarm()`` function lets us set an alarm for some number of seconds in the future. Passing ``0`` to the ``alarm()`` function cancels the alarm.

### Timeout in 5 seconds

We'll start by creating a decorator that times out in exactly 5 seconds, and then build from there to create a decorator that takes the timeout length as an argument. 

In [29]:
import signal

def timeout_in_5s(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Set an alarm for 5 seconds
        signal.alarm(5)
        try:
            # Call the decorated func
            return func(*args, **kwargs)
        finally:
            # Cancel alarm
            signal.alarm(0)
    return wrapper
        

Our ``timeout_in_5s()`` decorator starts off by defining a ``wrapper()`` function to return as the new decorated function.

Returning this function is what makes ``timeout_in_5s()`` a *decorator*. First ``wrapper()`` sets an alarm for 5 seconds in the future. 

Then it calls the function being decorated. It wraps that call in a try block so that in a finally block we can cancel the alarm. This ensures that the alarm either rings or gets canceled.

*Remember, when the alarm rings, Python calls the ``raise_timeout()`` function.*

Let's use ``timeout_in_5s()`` to decorate a function that **will definitely timeout**. 

In [ ]:
@timeout_in_5s
def foo():
    time.sleep(10)
    print('Foo')
    
foo()    

``foo()`` sleeps for 10 seconds and then prints ``"foo!"``. If we call ``foo()``, the 5-second alarm will ring before it finishes sleeping, and Python will raise a ``TimeoutErrror``.

Now let's create a more useful version of the ``timeout()`` decorator. This decorator takes an argument. To decorate ``foo()`` we'll set the timeout to 5 seconds like we did previously. But when decorating ``bar()``, we can set the timeout to 20 seconds. 

This allows us to set a timeout that is appropriate for each function. ``timeout()`` is a function that returns a ``decorator``. I like to think of it as a decorator factory. 

When you call ``timeout()``, it cranks out a brand new decorator that times out in 5 seconds, or 20 seconds, or whatever value we pass as ``n_seconds``.
 
 ```python
    def timeout(n_seconds):
        def decorator(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                # Set an alarm for n_seconds
                signal.signal(signal.SIGALRM, raise_timeout)
                signal.alarm(n_seconds)
                try:
                    # Call the function being decorated
                    return func(*args, **kwargs)
                finally:
                    # Cancel the alarm
                    signal.alarm(0)
                return wrapper
            return decorator        
 ```

The first thing we need to do is define this new decorator that it will return. That decorator begins, like all of our decorators, by defining a ``wrapper()`` function to return. 

Now because ``n_seconds`` is available to the ``wrapper()`` function we can set an alarm for ``n_seconds`` in the future. 

The rest of the ``wrapper()`` function looks exactly like the ``wrapper()`` function from the ``timeout_in_5s()`` function. Notice that ``wrapper()`` returns the result of calling ``func()``, ``decorator()`` returns *wrapper*, and ``timeout()`` returns *decorator*. 

```python
    @timeout(5)
    def foo():
        time.sleep(10)
        print("foo!")

    @timeout(20)
    def bar():
        time.sleep(10)
        print("bar!")

    # foo() will raise a TimeoutError
    # bar() will print "bar!"
```

So when we call ``foo()``, which has a 5-second timeout, it will timeout like before. But ``bar()``, which has a 20-second timeout, prints its message in 10 seconds, so the alarm gets canceled.